<a href="https://colab.research.google.com/github/jsansao/dlvc/blob/main/Tarefa7_SegmentacaoPets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarefa 7 - Segmentação do dataset Pets

Nessa tarefa, vamos observar o funcionamento do método U-net no dataset "Pets".  Notebook retirado do repositório [https://github.com/jakeret/unet](https://github.com/jakeret/unet). 


1. Rode o notebook com as configurações atuais; 
2. Opcional - Modifique a profundidade da rede convolucional (em `layer_depth`) e compare o desempenho das métricas e tempo de treinamento. 



In [ ]:
!pip install git+https://github.com/jakeret/unet.git

In [2]:
%load_ext tensorboard
import unet
from unet.datasets import oxford_iiit_pet
from tensorflow.keras import losses, metrics

In [5]:
LEARNING_RATE = 1e-3
unet_model = unet.build_model(*oxford_iiit_pet.IMAGE_SIZE,
                              channels=oxford_iiit_pet.channels,
                              num_classes=oxford_iiit_pet.classes,
                              layer_depth=5,
                              filters_root=64,
                              padding="same"
                              )

unet.finalize_model(unet_model,
                    loss=losses.SparseCategoricalCrossentropy(),
                    metrics=[metrics.SparseCategoricalAccuracy()],
                    auc=False,
                    learning_rate=LEARNING_RATE)


In [ ]:
train_dataset, validation_dataset = oxford_iiit_pet.load_data()

In [ ]:
%tensorboard --logdir oxford_iiit_pet

In [ ]:
trainer = unet.Trainer(name="oxford_iiit_pet", checkpoint_callback=False)
trainer.fit(unet_model,
            train_dataset,
            validation_dataset,
            epochs=25,
            batch_size=32)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


In [ ]:
rows = 10
fig, axs = plt.subplots(rows, 3, figsize=(8, 30))
for ax, (image, label) in zip(axs, train_dataset.take(rows).batch(1)):
  
  prediction = unet_model.predict(image)
  ax[0].matshow(image[0])
  ax[1].matshow(label[0, ..., 0], cmap="gray")
  ax[2].matshow(prediction[0].argmax(axis=-1), cmap="gray")